<a href="https://colab.research.google.com/github/anish2cod/NerualNetwork/blob/main/neuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
np.poly1d([1,0,1])


poly1d([1, 0, 1])

In [21]:
np.random.normal(size = (10,2))

array([[-0.19857181,  2.39847493],
       [ 1.21658491, -0.92526387],
       [-0.38603463,  1.165796  ],
       [-0.53754909,  0.80510569],
       [-2.75969271,  0.00575862],
       [-0.46520168,  0.50297423],
       [ 1.2008395 ,  0.23085788],
       [-0.47396705,  0.1543478 ],
       [ 0.29562354, -0.07362469],
       [ 1.77026343,  1.08750753]])

In [43]:
np.random.normal(size = (3 ,4)).T

array([[ 0.69632078, -0.33451948,  1.13469603],
       [ 1.5397437 ,  0.28863741,  0.77685656],
       [-0.3912268 , -0.20181201, -0.28924476],
       [-0.21034399, -0.7000741 ,  1.61014203]])

In [ ]:
# class Neuron:
#   def __init__(self, value = 0):
#     self.value = value


In [159]:
class Functions:
  # defining relu and its derivative
  def Relu(self,x):
    if x > 0:
      return x
    else:
      return 0
  def Relu_diff(self,x):
    if x > 0:
      return 1
    elif x < 0:
      return 0
    else:
      return 1/2

  def Leaky_Relu(self,x):
    if x >= 0:
      return x
    elif x < 0:
      return .15*x

  def Leaky_Relu_Diff(self,x):
    if x > 0 :
      return 1
    elif x<0:
      return .15
    else:
      return .125

  def SoftMax(self, Z):
    ls = np.array([np.exp(i) for i in Z])
    return ls/sum(ls)

  #defining the active
  def Activation(self,x,activation):
    if activation == "relu":
      return self.Relu(x)
    if activation == "leaky":
      return self.Leaky_Relu(x)
    if activation is None:
      return x

  def Diff(self,x,activation):
    if activation == "relu":
      return self.Relu_diff(x)
    if activation == "leaky":
      return self.Leaky_Relu_Diff(x)
    if activation is None:
      return 1

In [5]:
def SoftMax(Z):
  ls = np.array([np.exp(i) for i in Z])
  return ls/sum(ls)
ls = SoftMax(np.array([1,2,3,4]))
print(ls)

[0.0320586  0.08714432 0.23688282 0.64391426]


In [192]:
class Functions:
    Relu = lambda self, x: max(0, x)
    Relu_diff = lambda self, x: 1 if x > 0 else (0.5 if x == 0 else 0)

    Leaky_Relu = lambda self, x: x if x >= 0 else 0.15 * x
    Leaky_Relu_Diff = lambda self, x: 1 if x > 0 else (0.125 if x == 0 else 0.15)

    def Activation(self, x, activation):
        activations = {
            "relu": self.Relu,
            "leaky": self.Leaky_Relu
        }
        return activations.get(activation, lambda x: None)(x)

    def Diff(self, x, activation):
        derivatives = {
            "relu": self.Relu_diff,
            "leaky": self.Leaky_Relu_Diff
        }
        return derivatives.get(activation, lambda x: None)(x)


In [193]:
class Layer(Functions):
  def __init__(self, number_of_neurons,input_size , activation='relu',dropout=0,weight_init = None):

    self.neurons = None
    #random initilization need to be changed
    self.weights = np.random.normal(size = (number_of_neurons , input_size))
    self.bias = np.random.normal(size = number_of_neurons)
    #need to change
    self.activation = activation
    self.weight_init = weight_init
    self.dropout = dropout


  def FeedForward_L(self,input):
    self.pre_activation = self.weights @ input + self.bias
    self.neurons = np.array([self.Activation(x,self.activation) for x in self.pre_activation])
    return self.neurons

  def BackPropagation_L(self,grad,input):
    grad = grad * np.array([self.Diff(x,self.activation) for x in self.pre_activation])
    self.grad_bias = grad
    self.grad_weights = grad.reshape(-1,1) @ input.reshape(1,-1)
    grad = self.weights.T @ grad
    return grad


In [194]:
np.array([1,2,3]).reshape(-1,1) @ np.array([2,3,5]).reshape(1,-1)

array([[ 2,  3,  5],
       [ 4,  6, 10],
       [ 6,  9, 15]])

In [195]:
l1 = Layer(2,1,'relu',.3)
neurons1 = l1.FeedForward_L(np.array([1]))
neurons2 = l1.BackPropagation_L(np.array([1,2]),np.array([1]))
print("output " + str(neurons1))
print("weights" + str(l1.weights))
print("bias" + str(l1.bias))
print("grad_next" + str(neurons2))
print("grad_bias" + str(l1.grad_bias))
print("grad_weights" + str(l1.grad_weights))

output [0.         1.76546682]
weights[[-0.8348427 ]
 [ 1.53808598]]
bias[-0.13292656  0.22738084]
grad_next[3.07617196]
grad_bias[0 2]
grad_weights[[0]
 [2]]


In [196]:
l1.weights

array([[-0.8348427 ],
       [ 1.53808598]])

In [197]:
#to solve how to differentiate function using progamming otherwise mannually give the functions of derivatives

In [198]:
class Sequential:
  #stack the layers and their interactions
  def __init__(self,layers):
    self.layers = layers

  def FeedForward_S(self,input):
    for layer in self.layers:
      input = layer.FeedForward_L(input)
  def BackPropagation_S(self,grad,input1):
    for i in range(len(self.layers)-1,0,-1):
      grad = self.layers[i].BackPropagation_L(grad,self.layers[i-1].neurons)
    self.layers[0].BackPropagation_L(grad,input1)




In [207]:
seq = Sequential([
    Layer(2,3,"leaky"),
    Layer(1,2,"relu")
])


In [208]:
input = np.array([1000,2,30])
seq.FeedForward_S(input)
seq.BackPropagation_S(np.array([4]),input)
for layer in seq.layers:
  print(layer.grad_weights,layer.grad_bias)

[[ 1.46228824e+04  2.92457649e+01  4.38686473e+02]
 [-8.45134344e+02 -1.69026869e+00 -2.53540303e+01]] [14.62288244 -0.84513434]
[[ 670.57643427 -639.75272767]] [4]


In [173]:
seq

In [123]:
class NN:
  #fit,predict,backpropagation,feedforward etc
  #softmax contains all info for cross entropy

SyntaxError: incomplete input (<ipython-input-123-d1401462ada9>, line 2)